Mount your drive here

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
%cd '/content/drive/My Drive/Resonance'

/content/drive/My Drive/Resonance


In [11]:
%pwd

'/content/drive/My Drive/Resonance'

In [31]:
!git init

Initialized empty Git repository in /content/drive/My Drive/Resonance/Notebooks/.git/


In [65]:
!git clone https://github.com/jarmand-caw/projects.git

Cloning into 'projects'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [67]:
!git add .

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> projects
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached projects
hint: 
hint: See "git help submodule" for more information.


In [61]:
!git commit -m "Add existing file"
!git config --global user.email "jack.armand2@gmail.com"
!git config --global user.name "jarmand-caw"

On branch master
Changes not staged for commit:
	modified:   Data Intake.ipynb

no changes added to commit


In [54]:
!git remote add origin https://github.com/jarmand-caw/projects.git

fatal: remote origin already exists.


In [64]:
!git push origin master

fatal: could not read Username for 'https://github.com': No such device or address


In [30]:
%cd Notebooks

/content/drive/My Drive/Resonance/Notebooks


Import data from airtable

In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Resonance/Data.csv')

Functions used to split 'image' column from df above into seperate URLs for each image

In [0]:
import regex as re
def get_urls(s):
  found = re.findall(r'https.(.*?).jpg',s)
  found = ['https:'+x+'.jpg' for x in found]
  return found
def get_urls2(s):
  found = re.findall(r'\((.*)\)',s)
  found = [x for x in found if 'https' in x]
  return found
def get_urls3(s):
  s = s.split('https')[1]
  s = 'https'+s
  return s

Splitting image strings

In [0]:
df['Image URLS'] = df['IMAGE'].apply(lambda x: get_urls(x))
df = df.explode('Image URLS')
df['Image Name'] = [x for x in range(len(df))]

more = df.loc[df['Image URLS'].isna()].copy()
more['Image URLS'] = more['IMAGE'].apply(lambda x: get_urls2(x))
more['Image URLS'] = more['Image URLS'].apply(lambda x: get_urls3(x[0]))
for i in list(more.index):
  df['Image URLS'].at[i] = more['Image URLS'].at[i]
df['Image URLS'].at[83]

'https://dl.airtable.com/PktgMoVMTb665r9DVSDl_K56oeI4YTBmChk8TsW5U_Wrap-Kilt-Skirt-Navy-Stripe-wRust-Primrose-1'

Function to get the image from airtable. Random user agent so that airtable doesn't time me out for calling too many times

In [0]:
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
import requests
import shutil

def image(link,name):
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   
    user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=10)
    user_agent = user_agent_rotator.get_random_user_agent()
    referer = 'https://dl.airtable.com'
    headers = {'user-agent': user_agent,'referer':referer,'Authorization': 'Bearer keyKczEAXXRXR21i2'}
    response = requests.get(link, stream=True,headers=headers)
    response.raw.decode_content=True
    lfile = open('/content/drive/My Drive/Resonance/Images/'+name+'.jpg','wb')
    shutil.copyfileobj(response.raw,lfile)

Call image on each URL

In [0]:
import numpy as np

for i in range(len(df)):
  image(df['Image URLS'].iat[i],'Num.'+str(df['Image Name'].iat[i]))

Save df

In [0]:
df.to_csv('/content/drive/My Drive/Resonance/OriginalImageData.csv')

# ALL OF BELOW IS USED TO GET THE TRAINING DATA FOR EACH NEURAL NET

TAGS: Material

In [0]:
df['TAGS: Material'] = df['TAGS: Material'].fillna(0)
need_tags = df.loc[(df['TAGS: Material']==('None'))|(df['TAGS: Material']==0)]
have_tags = df.loc[~df.index.isin(list(need_tags.index))]

In [0]:
def step(idx):
  one = have_tags['TAGS: Material'].iat[idx]
  l = one.split(',')
  l2 = [x.strip() for x in l]
  l3 = [x.split(' ') for x in l2]
  one = pd.DataFrame(l3)
  one = one.groupby(0)[1].apply(list)
  one = pd.DataFrame(one)
  #print([str(have_tags['Image Name'].iat[idx])])
  #one = one.columns=[str(have_tags['Image Name'].iat[idx])]
  return one
for i in range(len(have_tags)):
  name = have_tags['Image Name'].iat[i]
  if i==0:
    all_df = step(i)
  else:
    all_df[name] = step(i).iloc[:,0]

In [0]:
import numpy as np

t = all_df.transpose()

type_names = list(set(t.explode('Type')['Type']))
type_names.remove(np.nan)

type_df = t.copy()
for i in type_names:
  type_df[i] = [np.nan for x in range(len(type_df))]
  for x in range(len(type_df)):
    if type(type_df['Type'].iat[x])==list:
      if i in type_df['Type'].iat[x]:
        type_df[i].iat[x] = 1

In [0]:
type_df = type_df.loc[~type_df['Type'].isnull()]
len(type_df)

1442

In [0]:
type_df.transpose().reset_index().iloc[4:].reset_index(drop=True).iloc[:,:1].to_csv('/content/drive/My Drive/Resonance/TypeCodeChart.csv')

In [0]:
type_df[type_names].fillna(0).to_csv('/content/drive/My Drive/Resonance/TypeData.csv')

In [0]:
pd.read_csv('/content/drive/My Drive/Resonance/TypeData.csv').transpose().iloc[1:].reset_index().iloc[:,:1].to_csv('/content/drive/My Drive/Resonance/TypeCodeChart.csv')

In [0]:
pd.read_csv('/content/drive/My Drive/Resonance/TypeData.csv').transpose().iloc[1:].reset_index().iloc[:,:1]

,index
0,Sheer
1,Jersey
2,Georgette
3,Twill
4,Plain
5,Crepe
6,Terry
7,Double
8,Flannel


In [0]:
t = all_df.transpose()
t['Chief'] = t['Chief'].apply(lambda x: x[0])
t['Chief Code'] = t['Chief'].astype('category').cat.codes
t['Chief Code'].to_csv('/content/drive/My Drive/Resonance/ChiefData.csv',header=False)

In [0]:
d = {}
for i in list(set(t['Chief'])):
  d[i]=set(t.loc[t['Chief']==i]['Chief Code'])
pd.DataFrame(pd.DataFrame(d).transpose()[0].apply(lambda x: list(x)[0])).to_csv('/content/drive/My Drive/Resonance/ChiefCodeChart.csv')

TAGS: Body

In [0]:
df['TAGS: Body'] = df['TAGS: Body'].fillna(0)
need_tags = df.loc[(df['TAGS: Body']==('None'))|(df['TAGS: Body']==0)]
have_tags = df.loc[~df.index.isin(list(need_tags.index))]

In [0]:
from IPython.core.debugger import set_trace

def step(idx):
  one = have_tags['TAGS: Body'].iat[idx]
  l = one.split(',')
  l2 = [x.strip() for x in l]
  l3 = [x.split(' ') for x in l2]
  one = pd.DataFrame(l3)
  one = one.groupby(0)[1].apply(list)
  one = pd.DataFrame(one)
  #print([str(have_tags['Image Name'].iat[idx])])
  #one = one.columns=[str(have_tags['Image Name'].iat[idx])]
  return one
for i in range(len(have_tags)):
  name = have_tags['Image Name'].iat[i]
  if i==0:
    all_df = step(i)
  else:
    all_df[name] = step(i).iloc[:,0]

In [0]:
t = all_df.transpose()
t['Len'] = t['Body'].apply(lambda x: len(x))
t['Body'] = t['Body'].apply(lambda x: x[0])
t['Body Code'] = t['Body'].astype('category').cat.codes
t['Body Code'].to_csv('/content/drive/My Drive/Resonance/BodyData.csv',header=False)

In [0]:
set(t['Body Code'])

{0, 1, 2, 3, 4}

In [0]:
d = {}
for i in list(set(t['Body'])):
  d[i]=set(t.loc[t['Body']==i]['Body Code'])

In [0]:
pd.DataFrame(pd.DataFrame(d).transpose()[0].apply(lambda x: list(x)[0])).to_csv('/content/drive/My Drive/Resonance/BodyCodeChart.csv')

In [0]:
import numpy as np

use_names = list(set(t.explode('Use')['Use']))
#use_names.remove(np.nan)

use_df = t.copy()
for i in use_names:
  use_df[i] = [np.nan for x in range(len(use_df))]
  for x in range(len(use_df)):
    if type(use_df['Use'].iat[x])==list:
      if i in use_df['Use'].iat[x]:
        use_df[i].iat[x] = 1

In [0]:
use_df[use_names].fillna(0).to_csv('/content/drive/My Drive/Resonance/UseData.csv')

In [0]:
pd.read_csv('/content/drive/My Drive/Resonance/UseData.csv').transpose().iloc[1:].reset_index()['index'].to_csv('/content/drive/My Drive/Resonance/UseCodeChart.csv',header=True)

In [0]:
use_df[use_names].transpose().reset_index().iloc[:,:1].to_csv('/content/drive/My Drive/Resonance/UseCodeChart.csv')

TAGS: Color

In [0]:
df['TAGS: Color'] = df['TAGS: Color'].fillna(0)
need_tags = df.loc[(df['TAGS: Color']==('None'))|(df['TAGS: Color']==0)]
have_tags = df.loc[~df.index.isin(list(need_tags.index))]

In [0]:
def step(idx):
  one = have_tags['TAGS: Color'].iat[idx]
  l = one.split(',')
  l2 = [x.strip() for x in l]
  l3 = [x.split(' ') for x in l2]
  one = pd.DataFrame(l3)
  one = one.groupby(0)[1].apply(list)
  one = pd.DataFrame(one)
  one = one.rename(columns={1:have_tags['Image Name'].iat[idx]})
  return one
for i in range(len(have_tags)):
  name = have_tags['Image Name'].iat[i]
  if i==0:
    all_df = step(i)
  else:
    all_df[name] = step(i)[name]

In [0]:
t = all_df.transpose()
#all_df['Primary'] = all_df['Primary'].apply(lambda x: x[0])
def flatten(x):
  if type(x)==list:
    return x[0]
  else:
    return x
t['Primary'] = t['Primary'].apply(lambda x: flatten(x))

In [0]:
import numpy as np
t = all_df.transpose()
genre_names = list(set(t.explode('Genre')['Genre']))
genre_names.remove(np.nan)

genre_df = t.copy()
for i in genre_names:
  genre_df[i] = [np.nan for x in range(len(genre_df))]
  for x in range(len(genre_df)):
    if type(genre_df['Genre'].iat[x])==list:
      if i in genre_df['Genre'].iat[x]:
        genre_df[i].iat[x] = 1


In [0]:
genre_df.loc[~genre_df['Genre'].isnull()].fillna(0)[genre_names].to_csv('/content/drive/My Drive/Resonance/GenreData.csv')

In [0]:
pd.DataFrame(genre_df.loc[~genre_df['Genre'].isnull()].fillna(0)[genre_names].transpose().index).to_csv('/content/drive/My Drive/Resonance/GenreCodeChart.csv')

In [0]:
primary_data_set = all_df.loc[~all_df['Primary'].isnull()]['Primary']

In [0]:
primary_data_set = pd.DataFrame(primary_data_set)
primary_data_set['Primary'] = primary_data_set['Primary'].astype('category')
primary_data_set['Primary_Code'] = primary_data_set['Primary'].cat.codes
set(primary_data_set['Primary_Code'])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}

In [0]:
d = {}
for i in list(set(all_df['Primary'])):
  d[i]=set(primary_data_set.loc[primary_data_set['Primary']==i]['Primary_Code'])

In [0]:
primary_data_set['Primary_Code'].to_csv('/content/drive/My Drive/Resonance/PrimaryData.csv',header=True)

In [0]:
import numpy as np

del d[np.nan]

In [0]:
codes = pd.DataFrame(d).transpose()
codes[0] = codes[0].apply(lambda x: list(x)[0])
codes.to_csv('/content/drive/My Drive/Resonance/PrimaryColorCodes.csv')